# Homework 1
## by Vo Ngoc Bich Uyen
## for the course "Models of sequential data"

We will consider the estimation for MA model for a particular dataset.
There are 5 problems in this homework.
* The first 4 problems are obligatory. 
* You can earn bonus points by making progress on the problem 5.

You have two weeks to complete the homework. The strict deadline is **17:00 $7$th of December, Moscow time**. 
No points will be awarded for the homework submitted after the deadline.


The name of the notebook should be in format *msd_hw_1_xxx.ipynb*, where *xxx* is your last name. E.g. *msd_hw_1_zaytsev.ipynb*.
Homeworks that fail to meet these guidelines, would have zero points as the grade.

## Problems 1-3. Estimation for MA models

### Problem 1. [10 points] 
Write a python class that defines ARMA model. 
* Problem 1.1. [5 points] Define the *initialize* method to initialize parameters of the model given orders for $ARMA(p, q)$
* Problem 1.2. [5 points] Define the *predict* method that can use ARMA to predict for one step and for $k$ steps ahead given the parameters of the model

### Problem 2. [15 points] 
Write a python method to *fit* the MA model of order $q$. 
I suggest to use the Least Squares method from the book "Time Series Analysis with Applications in R", Section 7.2 Least Squares Estimation. The book is freely available on the internet.

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)

In [31]:
class ARMA:
  #1.1. Define the initialize method to initialize parameters of the model given orders for ARMA(p,q)
  def __init__(self, p, q):
    self.p = p
    self.q = q
    self.ar_para = np.random.normal(0, 0.1, p)
    self.ma_para = np.random.normal(0, 0.1, q)

  def set_para (self, ar_para, ma_para):
    assert len(ar_para) == self.p, "Number of AR parameters are not equal p"
    assert len(ma_para) == self.q, "Number of MA parameters are not equal q"

    self.ar_para = ar_para
    self.ma_para = ma_para
    
  #1.2. Define the predict method for one step and for k steps ahead given the parameters of the model
  def predict(self, M, k):
    n = len(M)
    M = np.hstack([np.zeros(self.p), M, np.zeros(self.q)])
    errors = np.zeros(self.q + n + k)
    for i in range (n):
      ar_pred = np.dot(self.ar_para, M[i:i+self.p][::-1]) if self.p > 0 else 0
      ma_pred = np.dot(self.ma_para, errors[i:i+self.q][::-1]) if self.q > 0 else 0
      errors[i+self.q] = M[i+self.p] - ar_pred - ma_pred
      M = M[-(k+self.p):]
      errors = errors[-(k+self.q):]
      for i in range(k):
        ar_pred = np.dot(self.ar_para, M[i:i+self.p][::-1]) if self.p > 0 else 0
        ma_pred = np.dot(self.ma_para, errors[i:i+self.q][::-1]) if self.q > 0 else 0
        M[i+self.p] = ar_pred + ma_pred
      return M[self.p:]

    #2. Method to fit the MA model of order  q
  def fit_ma(self, M):
    def MSE(ma_para, M, q):
        errors = np.zeros(q+len(M))
        for i in range(len(M)):
          errors[i+q] = M[i] - errors[i:i+q][::-1] @ ma_para
        return np.sum(errors**2)
    mini = minimize(MSE, np.ones(q)*0.01, args = (M,self.q), method = "SLSQP")
    ma_para = mini.x
    self.ma_para = ma_para
    return ma_para

### Problem 3. [15 points] 
Validate the *fit* method you wrote
* Problem 3.1. [5 points] Write a procedure that generates a sequense for $MA(p)$ process with pre-defined parameters
* Problem 3.2. [10 points] Fit it using your methods and check the quality of forecasting and the quality of the parameter estimation. You can use scatter plots and MSE error for validation in this case.

In [32]:
# 3.1. Generate a sequense for MA(p) process with pre-defined parameters
def seq_ma(n, ma_para, q):
  errors = np.concatenate([np.zeros(q), np.random.normal(loc=0,scale=1,size=n)])
  M = np.array([errors[i+q] + np.dot(ma_para,errors[i:i+q][::-1]) for i in range(1,n)])
  return M

In [41]:
#3.2. Fit it using your methods and check the quality of forecasting and the quality of the parameter estimation.
from statsmodels.tsa.arima.model import ARIMA

q = 3
ma_para = np.array([1,0.9,0.3])

n_experiments = 10
n_seq = 1000

test_size = q
k = q

errors_para_self = np.zeros([n_experiments, q])
errors_para_stat = np.zeros([n_experiments, q])

tests = np.zeros([n_experiments, k])
pred_self = np.zeros([n_experiments, k])
pred_stat = np.zeros([n_experiments, k])

for i in range(n_experiments):
  M = seq_ma(n_seq, ma_para, q)
  M_train, M_test = M[:-test_size], M[-test_size:]

  model = ARMA(0, q)
  ma_pred_self = model.fit_ma(M_train)
  M_pred_self = model.predict(M_train, k = k)

  model = ARIMA(M_train, order=(0,0,q))
  results = model.fit()
  ma_pred_stat = results.polynomial_ma[1:]
  M_pred_stat = results.forecast(3)

  errors_para_self[i] = (ma_para - ma_pred_self) ** 2
  errors_para_stat[i] = (ma_para - ma_pred_stat) ** 2

  pred_self = M_pred_self
  pred_stat = M_pred_stat

  tests[i] = M_test[-k:]

print("MSE error of my prediction:", np.mean((pred_self-tests)**2))
print("MSE error of statsmodels prediction:", np.mean((pred_stat-tests)**2))

MSE error of my prediction: 2.1834637245987705
MSE error of statsmodels prediction: 2.130118371742841


## Problem 4.  Stationarity for AR process [10 points]
It is known, that the AR(p) process defined by the equation of the form 
$$
y_t = \phi_1 y_{t - 1} + \phi_2 y_{t - 2} + \ldots + \phi_p y_{t - p}.
$$
is weakly stationary if only if all roots of the regular characteristic polynomial have absolute values smaller than $1$.
The  regular characteristic polynomial has the form:
$$
R(\lambda) = \lambda^p - \phi_1 \lambda^{p - 1} - \ldots - \phi_p.
$$
Its roots are $\lambda_i$, $ i = 1, \ldots, p$. So, $\forall{i} \,\,  R(\lambda_i) = 0$.

Prove:
* Problem 4.1. [5 points] *Necessary condition:* If the process $AR(p)$ is weakly stationary, then $\sum_{i = 1}^p \phi_i < 1$.
* Problem 4.2. [5 points] *Sufficient condition:* If $\sum_{i = 1}^p |\phi_i| < 1$, then the corresponding process $AR(p)$ is weakly stationary.

**Problem 4.1.**
+ In case p = 1:
$$ y_t = \phi_1 y_{t - 1} $$
$$ R(\lambda) = \lambda - \phi_1 ⇒ R(\lambda_1) = \lambda_1 - \phi_1 = 0 ⇒ \lambda_1 = \phi_1 $$
The process $AR(1)$ is weakly stationary, so $|\lambda_1| < 1 ⇒ |\phi_1| < 1$ \\
+ In case p = 2:
$$ y_t = \phi_1 y_{t - 1} + \phi_2 y_{t - 2} $$
$$ R(\lambda) = \lambda^2 - \phi_1\lambda - \phi_2 $$
$$ ⇒ R(\lambda_i) = 0 ⇔ \delta = \phi_1^2 + 4\phi_2 \geq 0 $$
$$-1 < \lambda_1 \leq \lambda_2 < 1 $$
Apply Vieta's formula for quadratics: 
$$ \lambda_1  \lambda_2 = - \phi_2 $$
$$ \lambda_1  + \lambda_2 = \phi_1 $$
$$⇒ \phi_1 + \phi_2 + \lambda_1  \lambda_2 = \lambda_1 + \lambda_2 $$
$$⇒ |\phi_1 + \phi_2 + \lambda_1  \lambda_2| = |\lambda_1 + \lambda_2| \leq |\lambda_1| + |\lambda_2| < 2 $$
$$⇒ -2 < \phi_1 + \phi_2 + \lambda_1  \lambda_2 < 2 $$
$$⇒ -2 - \lambda_1  \lambda_2 < \phi_1 + \phi_2 < 2 - \lambda_1  \lambda_2 $$
Because $|\lambda_1\lambda_2| < 1 ⇒ -1 < -\lambda_1\lambda_2 \leq \lambda_1\lambda_2 < 1$
$$⇒ -1 < \phi_1 + \phi_2 < 1 $$
$$⇒ |\phi_1 + \phi_2| < 1 $$
+ In case p > 2: \\

**Problem 4.2.**
+ In case p = 1:
$$ y_t = \phi_1 y_{t - 1} $$
$$ R(\lambda) = \lambda - \phi_1 ⇒ R(\lambda_1) = \lambda_1 - \phi_1 = 0 ⇒ \lambda_1 = \phi_1 $$
If $|\phi_1| < 1 ⇒ |\lambda_1| < 1$, therefore the process $AR(1)$ is weakly stationary.
+ In case p = 2:
$$ y_t = \phi_1 y_{t - 1} + \phi_2 y_{t - 2} $$
$$ R(\lambda) = \lambda^2 - \phi_1\lambda - \phi_2 $$
$$ ⇒ R(\lambda_i) = 0 ⇔ \delta = \phi_1^2 + 4\phi_2 \geq 0 $$
$$\lambda_1 = \frac{\phi_1 - \sqrt{\phi_1^2 + 4\phi_2}}{2}$$
$$\lambda_2 = \frac{\phi_1 + \sqrt{\phi_1^2 + 4\phi_2}}{2}$$
We have $|\phi_1 + \phi_2| < 1 ⇒ -1 - \phi_1 < \phi_2 < 1 -\phi_1$
$$\delta = \phi_1^2 + 4\phi_2 <  \phi_1^2 + 4(1 -\phi_1) = (\phi_1 - 2)^2$$
$$⇒ \lambda_2 < \frac{\phi_1 - \phi_1 + 2}{2} = 1$$
$$~~~~~~ \lambda_1 > \frac{\phi_1 - \phi_1 + 2}{2} = 1$$
Because  $\lambda_1 \leq \lambda_2 ⇒ -1 < \lambda_1 \leq \lambda_2 < 1 ⇒ |\lambda_i| < 1, i= 1,2$, the process $AR(2)$ is weakly stationary.
+ In case p > 2: \\





## Problem 5. Gradient explosion* [20 points]

We consider a block in the recurrent neural network $h^t, x^t \in \mathbb{R}$:
$$
h^t = \phi(w_x x^t + w_h h^{t - 1} + b),
$$
where $\phi$ is the activation for the probit regression, CDF for the Gaussian distribution $\mathcal{N}(0, \sigma^2)$.
* Problem 5.1. [5 points] Find numerically or analytically (check, if possible) the expectation of the $\frac{\partial h^t}{\partial w_h}$ given $w_h \sim \mathcal{N}(0, \sigma^2)$.
* Problem 5.2. [5 points] Find numerically or analytically (check, if possible) the expectation of the $h^t$ given $w_h \sim \mathcal{N}(0, \sigma^2)$. In what cases we can get a stationary distribution for $h^t$?
* Problem 5.3. [10 points] Find the critical values for $\sigma^2$: in what case we'll have the explosion of the norm $\prod_{t = 1}^\infty \|\frac{\partial h^t}{\partial w_h} \| $? Provide numerical or analytical solution, if possible.